In [1]:
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [3]:
# Tabular modeling takes data in the form of a table.
# The objective is to predict the value in one column
# based on the values in the other columns.

In [4]:
# Continuous variables are numerical data such as 'age'
# Categorical variables contain a number of discrete levels.
# We can't perform numeric operations.

In [5]:
# Entity embedding not only reduces memeory usage and
# speeds up nn compared with one-hot encoding

In [6]:
# An Embedding layer is exactly equivalent to placing an
# ordinary linear layer after every one-hot input layer.

In [21]:
path = Path.cwd()/'datasets/bluebook'
path

Path('/home/fastai-2020-lalkrishna/My-files/datasets/bluebook')

In [22]:
df = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)
df.columnsd(2)

,SalesID,SalePrice,MachineID,ModelID,...,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,...,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,...,NaN,NaN,Standard,Conventional


In [23]:
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [24]:
df.ProductSize.unique()

array([nan, 'Medium', 'Small', 'Large / Medium', 'Mini', 'Large', 'Compact'], dtype=object)

In [25]:
sizes = 'Large', 'Large/Medium', 'Medium', 'Small', 'Mini', 'Compact'

In [33]:
df.ProductSize = df.ProductSize.astype('category')
df.ProductSize.cat.set_categories(sizes, ordered=True, inplace=True)

In [34]:
df.ProductSize.unique()

[NaN, 'Medium', 'Small', 'Mini', 'Large', 'Compact']
Categories (5, object): ['Large' < 'Medium' < 'Small' < 'Mini' < 'Compact']

In [35]:
# target column is "SalePrice"
# In kaggle, they useing RMSLE metrics

In [36]:
df.SalePrice = np.log(df.SalePrice)

## Decison tree

Loop through each column of the dataset in turn.

For each column, loop through each possible level of that column in turn.

Try splitting the data into two groups, based on whether they are greater than or less than that value (or if it is a categorical variable, based on whether they are equal to or not equal to that level of that categorical variable).

Find the average sale price for each of those two groups, and see how close that is to the actual sale price of each of the items of equipment in that group. That is, treat this as a very simple "model" where our predictions are simply the average sale price of the item's group.

After looping through all of the columns and all the possible levels for each, pick the split point that gave the best predictions using that simple model.

We now have two different groups for our data, based on this selected split. Treat each of these as separate datasets, and find the best split for each by going back to step 1 for each group.

Continue this process recursively, until you have reached some stopping criterion for each group—for instance, stop splitting a group further when it has only 20 items in it.

In [40]:
# fastai function add_datepart

In [41]:
df = add_datepart(df, 'saledate')

In [47]:
' '.join(o for o in df.columns if o.startswith('sale'))

'saleYear saleMonth saleWeek saleDay saleDayofweek saleDayofyear saleIs_month_end saleIs_month_start saleIs_quarter_end saleIs_quarter_start saleIs_year_end saleIs_year_start saleElapsed'

## TabularPandas and TabularProc

In [50]:
# TabularPandas -> Use for preprocessing to handle strings
# and missing data. alse split the data.

# TabularProcs is like a regular Transformer.
    # It return the same object, after modifying.

    # Categorify is a TabluarProc that replace a column with
      # numeric categorical columns
    
    # FillMissing -> replaces missing values with the median
      # creates a new boolean col that is set to True for any row 
      # where missing

In [51]:
procs = [Categorify, FillMissing]

In [54]:
cond = (df.saleYear < 2011) | (df.saleMonth<10)
train_idx = np.where(cond)[0]
valid_idx = np.where(~cond)[0]

In [62]:
splits = (list(train_idx), list(valid_idx))

In [63]:
# cont_cat_split -> return tuple, Continuous, categorical

In [65]:
cont, cat = cont_cat_split(df, 1, dep_var='SalePrice')

In [67]:
to = TabularPandas(df, procs, cat, cont, y_names='SalePrice', splits=splits)

In [69]:
to1 = TabularPandas(df, procs, ['state', 'ProductGroup', 'Drive_System', 'Enclosure'], [], y_names='SalePrice', splits=splits)

In [76]:
to.items.head(2)

,SalesID,SalePrice,MachineID,ModelID,...,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,11.097410,999089,3157,...,1,2647,1,1
1,1139248,10.950807,117657,77,...,1,2148,1,1


In [71]:
to1.show(2)

,state,ProductGroup,Drive_System,Enclosure,SalePrice
0,Alabama,WL,#na#,EROPS w AC,11.097410
1,North Carolina,WL,#na#,EROPS w AC,10.950807


In [79]:
to1.items[['state', 'ProductGroup', 'Drive_System', 'Enclosure']].head(2)

,state,ProductGroup,Drive_System,Enclosure
0,1,6,0,3
1,33,6,0,3


In [80]:
to.classes['ProductSize']

['#na#', 'Large', 'Large/Medium', 'Medium', 'Small', 'Mini', 'Compact']